<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork899-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# Peer Review Assignment - Data Engineer - ETL


Estimated time needed: **20** minutes


## Objectives

In this final part you will:

- Run the ETL process
- Extract bank and market cap data from the JSON file `bank_market_cap.json`
- Transform the market cap currency using the exchange rate data
- Load the transformed data into a seperate CSV


For this lab, we are going to be using Python and several Python libraries. Some of these libraries might be installed in your lab environment or in SN Labs. Others may need to be installed by you. The cells below will install these libraries when executed.


In [1]:
!mamba install pandas==1.3.3 -y
!mamba install requests==2.26.0 -y


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (0.27.0) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['pandas==1.3.3']

[+] 0.0s
opt/ibm/custom-channels/meta-wscloud/linux-64 ━━╸━━━━━━━━━━━━━━━╸━   0.

Extracting   (3) ━━━━╸━━━━━━━━━━━━━━━╸━━       0 openssl                    0.4s[+] 0.6s
Extracting   (3) ━━━━━╸━━━━━━━━━━━━━━━╸━       0 openssl                    0.5s[+] 0.7s
Extracting   (3) ━━━━━━╸━━━━━━━━━━━━━━━━       0 openssl                    0.6s[+] 0.8s
Extracting   (3) ━━━━━━━━╸━━━━━━━━━━━━━━       0 openssl                    0.7s[+] 0.9s
Extracting   (3) ━━━━━━━━━╸━━━━━━━━━━━━━       0 requests                   0.8s[+] 1.0s
Extracting   (3) ━━━━━━━━━━╸━━━━━━━━━━━━       0 requests                   0.9s[+] 1.1s
Extracting   (3) ━━━━━━━━━━━╸━━━━━━━━━━━       0 requests                   1.0s[+] 1.2s
Extracting   (3) ━━━━━━━━━━━━╸━━━━━━━━━━       0 requests                   1.1s[+] 1.3s
Extracting   (3) ━━━━━━━╸━━━━━━━━━━━━━━━       0 ibm-wsrt-py310t2-def       1.2s[+] 1.4s
Extracting   (3) ━━━━━━━━╸━━━━━━━━━━━━━━       0 ibm-wsrt-py310t2-def       1.3s[+] 1.5s
Extracting   (3) ━━━━━━━━━╸━━━━━━━━━━━━━       0 ibm-wsrt-py310t2-def       1.4s[+] 1.6s
Extracting   (3) ━━━━

## Imports

Import any additional libraries you may need here.


In [2]:
import glob
import pandas as pd
from datetime import datetime

As the exchange rate fluctuates, we will download the same dataset to make marking simpler. This will be in the same format as the dataset you used in the last section  


In [3]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv

--2023-06-19 01:44:27--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2815 (2.7K) [application/json]
Saving to: ‘bank_market_cap_1.json’

bank_market_cap_1.j 100%[===================>]   2.75K  --.-KB/s    in 0s      

2023-06-19 01:44:28 (27.8 MB/s) - ‘bank_market_cap_1.json’ saved [2815/2815]

--2023-06-19 01:44:30--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Tra

## Extract


### JSON Extract Function

This function will extract JSON files.


In [4]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

## Extract Function

Define the extract function that finds JSON file `bank_market_cap_1.json` and calls the function created above to extract data from them. Store the data in a `pandas` dataframe. Use the following list for the columns.


In [5]:
columns=['Name','Market Cap (US$ Billion)']

In [6]:
def extract():
    # Write your code here
    fname = "bank_market_cap_1.json"
    df = extract_from_json(fname)
    df2 = df[columns]
    return df2

<b>Question 1</b> Load the file <code>exchange_rates.csv</code> as a dataframe and find the exchange rate for British pounds with the symbol <code>GBP</code>, store it in the variable  <code>exchange_rate</code>, you will be asked for the number. Hint: set the parameter  <code>index_col</code> to 0.


In [7]:
# Write your code here
df_exr = pd.read_csv("exchange_rates.csv",index_col=0)
exchange_rate = df_exr.loc["GBP","Rates"]
print(df_exr.head())
print()
print("exchange_rate = ", exchange_rate)

        Rates
AUD  1.297088
BGN  1.608653
BRL  5.409196
CAD  1.271426
CHF  0.886083

exchange_rate =  0.7323984208000001


## Transform

Using <code>exchange_rate</code> and the `exchange_rates.csv` file find the exchange rate of USD to GBP. Write a transform function that

1. Changes the `Market Cap (US$ Billion)` column from USD to GBP
2. Rounds the Market Cap (US$ Billion)` column to 3 decimal places
3. Rename `Market Cap (US$ Billion)` to `Market Cap (GBP$ Billion)`


In [8]:
def transform(dataFrame, ex_rate):
    # Write your code here
    usd2gbp = dataFrame.loc["USD","Rates"] / ex_rate
    df_usd = extract()
    df_usd.loc[:, "Market Cap (GBP$ Billion)"] = round(df_usd.iloc[:,1] / usd2gbp, 3)
    df_gbp = df_usd.drop("Market Cap (US$ Billion)",axis=1)
    return df_gbp, usd2gbp

## Load

Create a function that takes a dataframe and load it to a csv named `bank_market_cap_gbp.csv`. Make sure to set `index` to `False`.


In [9]:
def load(data_frame, filename):
    # Write your code here
    data_frame.to_csv(filename, index=False)

## Logging Function


Write the logging function <code>log</code> to log your data:


In [10]:
def log(msg):
    # Write your code here
    with open("log.txt", "a") as f:
        dt = datetime.today().strftime("%Y-%m-%d %H:%M-%S")
        f.write(str(dt)+ " "+ str(msg) + "\n")

## Running the ETL Process


Log the process accordingly using the following <code>"ETL Job Started"</code> and <code>"Extract phase Started"</code>


In [11]:
# Write your code here
log("ETL Job Started")
log("Extrat phase Started")

### Extract


<code>Question 2</code> Use the function <code>extract</code>, and print the first 5 rows, take a screen shot:


In [12]:
# Call the function here
df0 = extract()
# Print the rows here
df0.head()

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399


Log the data as <code>"Extract phase Ended"</code>


In [14]:
# Write your code here
log("Extract phase Ended")

### Transform


Log the following  <code>"Transform phase Started"</code>


In [15]:
# Write your code here
log ("Transform phase Started")

<code>Question 3</code> Use the function <code>transform</code> and print the first 5 rows of the output, take a screen shot:


In [16]:
# Call the function here
df_gbp, usd2gbp = transform(df_exr, exchange_rate)
# Print the first 5 rows here
print(df_gbp.head())

                                      Name  Market Cap (GBP$ Billion)
0                           JPMorgan Chase                    286.319
1  Industrial and Commercial Bank of China                    252.834
2                          Bank of America                    238.272
3                              Wells Fargo                    225.588
4                  China Construction Bank                    188.519


Log your data <code>"Transform phase Ended"</code>


In [17]:
# Write your code here
log ("Transform phase Ended")

### Load


Log the following `"Load phase Started"`.


In [18]:
# Write your code here
log ("Load phase Started")

Call the load function


In [19]:
# Write your code here
load(df_gbp, "bank_market_cap_gbp.csv")

Log the following `"Load phase Ended"`.


In [20]:
# Write your code here
log("Load phase Ended")

## Authors


Ramesh Sannareddy, Joseph Santrcangelo and Azim Hirjani


### Other Contributors


Rav Ahuja


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By        | Change Description                 |
| ----------------- | ------- | ----------------- | ---------------------------------- |
| 2020-11-25        | 0.1     | Ramesh Sannareddy | Created initial version of the lab |


 Copyright © 2020 IBM Corporation. This notebook and its source code are released under the terms of the [MIT License](https://cognitiveclass.ai/mit-license?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork899-2023-01-01&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBM-DA0321EN-SkillsNetwork-21426264&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ).
